In [1]:
import openpathsampling as paths

In [2]:
# always the most complicated bit
import openpathsampling.engines.toy as toys
import numpy as np

pes = (toys.OuterWalls([1.0,1.0], [0.0,0.0]) + 
       toys.Gaussian(-0.7, [12.0, 0.5], [-0.5, 0.0]) +
       toys.Gaussian(-0.7, [12.0, 0.5], [0.5, 0.0]))
topology = toys.Topology(n_spatial=2, masses=[1.0, 1.0], pes=pes)
template = toys.Snapshot(coordinates=np.array([[0.0, 0.0]]),
                         velocities=np.array([[0.0, 0.0]]),
                         topology=topology)
engine = toys.Engine(options={'integ': toys.LangevinBAOABIntegrator(dt=0.02, temperature=0.1, gamma=2.5),
                              'n_frames_max': 5000,
                              'n_steps_per_frame': 10},
                     template=template)

In [3]:
# states are volumes in a CV space: define the CV
def xval(snapshot):
    return snapshot.xyz[0][0]

cv = paths.FunctionCV("xval", xval)

In [4]:
stateA = paths.CVDefinedVolume(cv, float("-inf"), -0.2).named("A")
stateB = paths.CVDefinedVolume(cv, 0.2, float("inf")).named("B")

In [5]:
network = paths.FixedLengthTPSNetwork(stateA, stateB, length=5000)

In [6]:
scheme = paths.OneWayShootingMoveScheme(network, engine=engine)

In [7]:
# I'll fake an initial trajectory
delta = 2.0 / 5000
minimum = -1.0 - delta
trajectory = paths.Trajectory([
    toys.Snapshot(coordinates=np.array([[minimum+k*delta, 0.0]]),
                  velocities=np.array([[0.1, 0.0]]),
                  topology=topology)
    for k in range(5000)
])

In [8]:
initial_conditions = paths.SampleSet.map_trajectory_to_ensembles(trajectory, network.sampling_ensembles)

In [10]:
storage = paths.Storage("simple_fixedlen.nc", "w", template=template)
simulation = paths.PathSampling(storage, scheme, initial_conditions)

In [11]:
simulation.run(200)

Working on Monte Carlo cycle number 200.

DONE! Completed 200 Monte Carlo cycles.

